In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json 
from pandas.io.json import json_normalize

In [34]:
businesses = pd.read_csv('Business_Licenses.csv')

In [3]:
businesses.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,22308-20060816,1723393,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,...,08/16/2006,08/15/2007,08/10/2006,08/11/2006,AAI,NaN,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)"
1,1620668-20160516,2455262,295026,1,BUCCI BIG & TALL INC.,BUCCI BIG & TALL INC.,558 W ROOSEVELT RD,CHICAGO,IL,60607,...,05/16/2016,05/15/2018,05/18/2016,08/30/2016,AAI,NaN,NaN,41.867339,-87.641590,"(41.86733856638269, -87.64159005699716)"
2,2368602-20160616,2460909,291461,3,"PROJECT: VISION , INC.","PROJECT : VISION , INC",2301 S ARCHER AVE 1 1,CHICAGO,IL,60616,...,06/16/2016,06/15/2017,06/21/2016,06/22/2016,AAC,08/30/2016,NaN,41.850843,-87.638734,"(41.85084294374687, -87.63873424399071)"
3,2060891-20141016,2353257,357247,1,FOLASHADE'S CLEANING SERVICE INC.,FOLASHADE'S CLEANING SERVICE INC.,1965 BERNICE RD 1 1SW,LANSING,IL,60438,...,10/16/2014,10/15/2016,04/01/2016,04/01/2016,AAI,NaN,38.0,41.951316,-87.678586,"(41.95131555606832, -87.67858578019546)"
4,1144216-20070516,1804790,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,...,05/16/2007,05/15/2008,05/10/2007,05/11/2007,AAI,NaN,5.0,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)"


In [4]:
# clean latitude and longitude columns

#dropnans 
businesses = businesses[np.isfinite(businesses['LONGITUDE'])]
businesses = businesses[np.isfinite(businesses['LATITUDE'])]

In [5]:
#Change all dates to datetime format
businesses['LICENSE TERM START DATE'] = pd.to_datetime(businesses['LICENSE TERM START DATE'], format='%m/%d/%Y')
businesses['LICENSE TERM EXPIRATION DATE'] = pd.to_datetime(businesses['LICENSE TERM EXPIRATION DATE'], format='%m/%d/%Y')
businesses['APPLICATION CREATED DATE'] = pd.to_datetime(businesses['APPLICATION CREATED DATE'], format='%m/%d/%Y')
businesses['APPLICATION REQUIREMENTS COMPLETE'] = pd.to_datetime(businesses['APPLICATION REQUIREMENTS COMPLETE'], format='%m/%d/%Y')
businesses['PAYMENT DATE'] = pd.to_datetime(businesses['PAYMENT DATE'], format='%m/%d/%Y')
businesses['LICENSE APPROVED FOR ISSUANCE'] = pd.to_datetime(businesses['LICENSE APPROVED FOR ISSUANCE'], format='%m/%d/%Y')
businesses['DATE ISSUED'] = pd.to_datetime(businesses['DATE ISSUED'], format='%m/%d/%Y')
businesses['LICENSE STATUS CHANGE DATE'] = pd.to_datetime(businesses['LICENSE STATUS CHANGE DATE'], format='%m/%d/%Y')

In [6]:
businesses.size

30577254

In [7]:
businesses.columns

Index(['ID', 'LICENSE ID', 'ACCOUNT NUMBER', 'SITE NUMBER', 'LEGAL NAME',
       'DOING BUSINESS AS NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP CODE',
       'WARD', 'PRECINCT', 'WARD PRECINCT', 'POLICE DISTRICT', 'LICENSE CODE',
       'LICENSE DESCRIPTION', 'BUSINESS ACTIVITY ID', 'BUSINESS ACTIVITY',
       'LICENSE NUMBER', 'APPLICATION TYPE', 'APPLICATION CREATED DATE',
       'APPLICATION REQUIREMENTS COMPLETE', 'PAYMENT DATE',
       'CONDITIONAL APPROVAL', 'LICENSE TERM START DATE',
       'LICENSE TERM EXPIRATION DATE', 'LICENSE APPROVED FOR ISSUANCE',
       'DATE ISSUED', 'LICENSE STATUS', 'LICENSE STATUS CHANGE DATE', 'SSA',
       'LATITUDE', 'LONGITUDE', 'LOCATION'],
      dtype='object')

In [23]:
(~businesses['BUSINESS ACTIVITY'].isna()).sum()

418146

In [38]:
#Get rid of all nan values in the business dataframe
businesses = businesses.dropna()
#businesses = businesses['BUSINESS ACTIVITY'].dropna()

In [39]:
businesses.size

36210

In [9]:
data = []
with open("Neighborhoods_2012_polygons.json", "r") as json_data:
    data = json.load(json_data)
df = {}
for item in data["features"]:
    if  item["type"] =='Feature':
        if 'properties' in item.keys():
            nn = item.get("properties").get("PRI_NEIGH")
            df[nn] = nn
        if 'geometry' in item:
            coords = item.get('geometry').get('coordinates')
            df[nn] = coords
df_n=pd.DataFrame(pd.Series(df))
df_n.head()
#df_n = pd.DataFrame(pd.split(""))

,0
Grand Boulevard,"[[[-87.60670812560363, 41.81681377137387], [-8..."
Printers Row,"[[[-87.62760697485339, 41.87437097785366], [-8..."
United Center,"[[[-87.66706868914592, 41.88885187769542], [-8..."
Sheffield & DePaul,"[[[-87.65833494805524, 41.921661442291786], [-..."
Humboldt Park,"[[[-87.74059567509258, 41.88782316893226], [-8..."


In [10]:
df_new = pd.DataFrame({'Neighborhood': df_n.index.values, 'Polygons': df_n[0].values})
df_new.head()

,Neighborhood,Polygons
0,Grand Boulevard,"[[[-87.60670812560363, 41.81681377137387], [-8..."
1,Printers Row,"[[[-87.62760697485339, 41.87437097785366], [-8..."
2,United Center,"[[[-87.66706868914592, 41.88885187769542], [-8..."
3,Sheffield & DePaul,"[[[-87.65833494805524, 41.921661442291786], [-..."
4,Humboldt Park,"[[[-87.74059567509258, 41.88782316893226], [-8..."


In [11]:
df_new['Polygons_tup'] = df_new['Polygons'].apply(lambda x: [tuple(a) for a in x[0]])

In [12]:
df_new.head()

,Neighborhood,Polygons,Polygons_tup
0,Grand Boulevard,"[[[-87.60670812560363, 41.81681377137387], [-8...","[(-87.60670812560363, 41.81681377137387), (-87..."
1,Printers Row,"[[[-87.62760697485339, 41.87437097785366], [-8...","[(-87.62760697485339, 41.87437097785366), (-87..."
2,United Center,"[[[-87.66706868914592, 41.88885187769542], [-8...","[(-87.66706868914592, 41.88885187769542), (-87..."
3,Sheffield & DePaul,"[[[-87.65833494805524, 41.921661442291786], [-...","[(-87.65833494805524, 41.921661442291786), (-8..."
4,Humboldt Park,"[[[-87.74059567509258, 41.88782316893226], [-8...","[(-87.74059567509258, 41.88782316893226), (-87..."


In [13]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [83]:

def neighbor_finder(lat, long):
    for index,row in df_new.iterrows():
        poly = row['Polygons_tup']
        if len(poly) >= 3:
            polygon = Polygon(poly)
            point = Point(lat, long)
            if polygon.contains(point):
                neighborhood = row['Neighborhood']
    return neighborhood

In [41]:
long = -87.723960
lat = 41.825320
neighbor_finder(long, lat)

'Little Village'

In [84]:
nhoods = []
for index, row in businesses.iterrows():
    lat = row['LATITUDE']
    long = row['LONGITUDE']
    hood = neighbor_finder(long,lat)
    #print(hood)
    nhoods.append(hood)
    
    
businesses['Neighborhood'] = nhoods    

In [85]:
businesses.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION,Neighborhood
3775,2115351-20111128,2115351,366326,1,FARMA INC.,CLARK,11100 S STATE ST 1ST,CHICAGO,IL,60628,...,10/15/2013,11/28/2011,11/28/2011,AAC,05/03/2013,40.0,41.692537,-87.623249,"(41.692537041934436, -87.62324926529826)",Roseland
7256,2428252-20150925,2428252,28998,40,"PARAGON, PARADIGM, PARADOX, INC.",CHICAGO COSTUME,4704 N BROADWAY 1ST,CHICAGO,IL,60640,...,04/15/2017,09/25/2015,09/25/2015,AAC,09/02/2016,34.0,41.967298,-87.658852,"(41.967298486723465, -87.65885177420338)",Uptown
9483,2384888-20150406,2384888,396435,1,"NOOR 71, INC.",CITGO,2801-2813 W 71ST ST 1ST,CHICAGO,IL,60629,...,04/15/2017,04/03/2015,04/06/2015,AAC,10/05/2015,14.0,41.764429,-87.693086,"(41.76442886776059, -87.69308649820891)",Chicago Lawn
14225,2428683-20151020,2428683,401673,1,AYUB MOHAMED,HAWAM INTERNATIONAL TRADING,5238 N WINTHROP AVE 2ND 2E,CHICAGO,IL,60640,...,11/15/2017,10/20/2015,10/20/2015,AAC,09/06/2016,26.0,41.977518,-87.657993,"(41.97751801125337, -87.65799299715734)",Edgewater
16333,2374664-20150113,2374664,395783,1,TRUNG NGUYEN AND PHUONG LUONG,REVAMP NAILS,4717 N LINCOLN AVE,CHICAGO,IL,60625,...,01/15/2017,01/13/2015,01/13/2015,AAC,09/08/2016,21.0,41.967101,-87.687416,"(41.96710140995649, -87.68741615506828)",Lincoln Square


In [22]:
len(nhoods)

13151

In [75]:
businesses.columns

Index(['ID', 'LICENSE ID', 'ACCOUNT NUMBER', 'SITE NUMBER', 'LEGAL NAME',
       'DOING BUSINESS AS NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP CODE',
       'WARD', 'PRECINCT', 'WARD PRECINCT', 'POLICE DISTRICT', 'LICENSE CODE',
       'LICENSE DESCRIPTION', 'BUSINESS ACTIVITY ID', 'BUSINESS ACTIVITY',
       'LICENSE NUMBER', 'APPLICATION TYPE', 'APPLICATION CREATED DATE',
       'APPLICATION REQUIREMENTS COMPLETE', 'PAYMENT DATE',
       'CONDITIONAL APPROVAL', 'LICENSE TERM START DATE',
       'LICENSE TERM EXPIRATION DATE', 'LICENSE APPROVED FOR ISSUANCE',
       'DATE ISSUED', 'LICENSE STATUS', 'LICENSE STATUS CHANGE DATE', 'SSA',
       'LATITUDE', 'LONGITUDE', 'LOCATION', 'Neighborhood'],
      dtype='object')

In [ ]:
for i in businesses.index:
        val = businesses.loc[i,'Age']

In [ ]:
businesses['Neighborhood'] = np.nan

In [82]:
businesses.apply(neighbor_finder, axis=0, args = (businesses['LONGITUDE'], businesses['LATITUDE']))

TypeError: ('neighbor_finder() takes 2 positional arguments but 3 were given', 'occurred at index ID')

In [ ]:
len(nhoods)

In [ ]:
businesses.head()

In [ ]:
businesses['BUSINESS ACTIVITY'].head()

In [44]:
businesses['BUSINESS ACTIVITY'].value_counts()

Retail Sales of Perishable Foods                                                                                                                                                                                                                 97
Consumption of Liquor on Premises                                                                                                                                                                                                                85
Preparation of Food and Dining on Premise With Seating                                                                                                                                                                                           72
Retail Sales of General Merchandise                                                                                                                                                                                                              57
Nail Services           

In [ ]:
businesses['BUSINESS ACTIVITY'].isna().sum()

In [ ]:
(~businesses['BUSINESS ACTIVITY'].isna()).sum()

In [45]:
health = pd.read_csv('Public_Health_Statistics-_Selected_public_health_indicators_by_Chicago_community_area.csv')

In [46]:
health.head()

,Community Area,Community Area Name,Birth Rate,General Fertility Rate,Low Birth Weight,Prenatal Care Beginning in First Trimester,Preterm Births,Teen Birth Rate,Assault (Homicide),Breast cancer in females,...,Childhood Lead Poisoning,Gonorrhea in Females,Gonorrhea in Males,Tuberculosis,Below Poverty Level,Crowded Housing,Dependency,No High School Diploma,Per Capita Income,Unemployment
0,1,Rogers Park,16.4,62.0,11.0,73.0,11.2,40.8,7.7,23.3,...,0.5,322.5,423.3,11.4,22.7,7.9,28.8,18.1,23714,7.5
1,2,West Ridge,17.3,83.3,8.1,71.1,8.3,29.9,5.8,20.2,...,1.0,141.0,205.7,8.9,15.1,7.0,38.3,19.6,21375,7.9
2,3,Uptown,13.1,50.5,8.3,77.7,10.3,35.1,5.4,21.3,...,0.5,170.8,468.7,13.6,22.7,4.6,22.2,13.6,32355,7.7
3,4,Lincoln Square,17.1,61.0,8.1,80.5,9.7,38.4,5.0,21.7,...,0.4,98.8,195.5,8.5,9.5,3.1,25.6,12.5,35503,6.8
4,5,North Center,22.4,76.2,9.1,80.4,9.8,8.4,1.0,16.6,...,0.9,85.4,188.6,1.9,7.1,0.2,25.5,5.4,51615,4.5


In [47]:
health.columns

Index(['Community Area', 'Community Area Name', 'Birth Rate',
       'General Fertility Rate', 'Low Birth Weight',
       'Prenatal Care Beginning in First Trimester', 'Preterm Births',
       'Teen Birth Rate', 'Assault (Homicide)', 'Breast cancer in females',
       'Cancer (All Sites)', 'Colorectal Cancer', 'Diabetes-related',
       'Firearm-related', 'Infant Mortality Rate', 'Lung Cancer',
       'Prostate Cancer in Males', 'Stroke (Cerebrovascular Disease)',
       'Childhood Blood Lead Level Screening', 'Childhood Lead Poisoning',
       'Gonorrhea in Females', 'Gonorrhea in Males', 'Tuberculosis',
       'Below Poverty Level', 'Crowded Housing', 'Dependency',
       'No High School Diploma', 'Per Capita Income', 'Unemployment'],
      dtype='object')

In [48]:
len(health) - health.count()

Community Area                                 0
Community Area Name                            0
Birth Rate                                     0
General Fertility Rate                         0
Low Birth Weight                               0
Prenatal Care Beginning in First Trimester     0
Preterm Births                                 0
Teen Birth Rate                                0
Assault (Homicide)                             0
Breast cancer in females                       0
Cancer (All Sites)                             0
Colorectal Cancer                              0
Diabetes-related                               0
Firearm-related                                0
Infant Mortality Rate                          0
Lung Cancer                                    0
Prostate Cancer in Males                       0
Stroke (Cerebrovascular Disease)               0
Childhood Blood Lead Level Screening           1
Childhood Lead Poisoning                       1
Gonorrhea in Females

In [63]:
#Replace nan values with column averages
meanfemgon = health['Gonorrhea in Females'].mean()
health['Gonorrhea in Females'].fillna(meanfemgon, inplace=True)
meanbloodlevel = health['Childhood Blood Lead Level Screening'].mean()
health['Childhood Blood Lead Level Screening'].fillna(meanbloodlevel, inplace=True)
meanlead = health['Childhood Lead Poisoning'].mean()
health['Childhood Lead Poisoning'].fillna(meanlead, inplace=True)

In [67]:
deaths = pd.read_csv('Public_Health_Statistics-_Selected_underlying_causes_of_death_in_Chicago__2006___2010.csv')

In [68]:
deaths.head()

,Cause of Death,Community Area,Community Area Name,Cumulative Deaths 2006 - 2010,Cumulative Deaths Rank,Average Annual Deaths 2006 - 2010,Average Crude Rate 2006 - 2010,Average Crude Rate Lower CI,Average Crude Rate Upper CI,Crude Rate Rank,Average Adjusted Rate 2006 - 2010,Average Adjusted Rate Lower CI,Average Adjusted Rate Upper CI,Adjusted Rate Rank,Average Annual Years of Potential Life Lost (YPLL) Rate 2006 - 2010,YPLL Rate RANK,WARNING
0,All Causes,1,Rogers Park,1741,22.0,348,614.2,585.3,643.1,51.0,895.2,852.2,939.3,29.0,7128,37.0,NaN
1,All Causes,2,West Ridge,2612,5.0,522,723.6,695.8,751.4,42.0,689.9,663.3,717.1,58.0,6123,49.0,NaN
2,All Causes,3,Uptown,2542,6.0,508,879.6,845.4,913.8,31.0,934.0,897.5,971.2,24.0,9147,31.0,NaN
3,All Causes,4,Lincoln Square,1108,40.0,222,547.0,514.8,579.2,60.0,698.7,657.4,741.3,53.0,5344,62.0,NaN
4,All Causes,5,North Center,690,59.0,138,433.0,400.7,465.3,68.0,681.2,630.1,734.3,61.0,4058,74.0,NaN


In [69]:
deaths.columns

Index(['Cause of Death', 'Community Area', 'Community Area Name',
       'Cumulative Deaths 2006 - 2010', 'Cumulative Deaths Rank',
       'Average Annual Deaths 2006 - 2010', 'Average Crude Rate 2006 - 2010',
       'Average Crude Rate Lower CI', 'Average Crude Rate Upper CI',
       'Crude Rate Rank', 'Average Adjusted Rate 2006 - 2010',
       'Average Adjusted Rate Lower CI', 'Average Adjusted Rate Upper CI',
       'Adjusted Rate Rank',
       'Average Annual Years of Potential Life Lost (YPLL) Rate 2006 - 2010',
       'YPLL Rate RANK', 'WARNING'],
      dtype='object')

In [70]:
len(deaths) - deaths.count()

Cause of Death                                                           0
Community Area                                                           0
Community Area Name                                                      0
Cumulative Deaths 2006 - 2010                                            0
Cumulative Deaths Rank                                                  18
Average Annual Deaths 2006 - 2010                                        0
Average Crude Rate 2006 - 2010                                           0
Average Crude Rate Lower CI                                              0
Average Crude Rate Upper CI                                              5
Crude Rate Rank                                                         18
Average Adjusted Rate 2006 - 2010                                        0
Average Adjusted Rate Lower CI                                           5
Average Adjusted Rate Upper CI                                           5
Adjusted Rate Rank       

In [71]:
deaths

,Cause of Death,Community Area,Community Area Name,Cumulative Deaths 2006 - 2010,Cumulative Deaths Rank,Average Annual Deaths 2006 - 2010,Average Crude Rate 2006 - 2010,Average Crude Rate Lower CI,Average Crude Rate Upper CI,Crude Rate Rank,Average Adjusted Rate 2006 - 2010,Average Adjusted Rate Lower CI,Average Adjusted Rate Upper CI,Adjusted Rate Rank,Average Annual Years of Potential Life Lost (YPLL) Rate 2006 - 2010,YPLL Rate RANK,WARNING
0,All Causes,1,Rogers Park,1741,22.0,348,614.2,585.3,643.1,51.0,895.2,852.2,939.3,29.0,7128,37.0,NaN
1,All Causes,2,West Ridge,2612,5.0,522,723.6,695.8,751.4,42.0,689.9,663.3,717.1,58.0,6123,49.0,NaN
2,All Causes,3,Uptown,2542,6.0,508,879.6,845.4,913.8,31.0,934.0,897.5,971.2,24.0,9147,31.0,NaN
3,All Causes,4,Lincoln Square,1108,40.0,222,547.0,514.8,579.2,60.0,698.7,657.4,741.3,53.0,5344,62.0,NaN
4,All Causes,5,North Center,690,59.0,138,433.0,400.7,465.3,68.0,681.2,630.1,734.3,61.0,4058,74.0,NaN
5,All Causes,6,Lake View,1914,14.0,383,405.3,387.1,423.5,70.0,599.5,572.2,627.5,70.0,3281,76.0,NaN
6,All Causes,7,Lincoln Park,1424,30.0,285,443.9,420.8,467.0,67.0,698.7,662.0,736.5,53.0,2962,77.0,NaN
7,All Causes,8,Near North Side,2054,10.0,411,520.3,497.8,542.8,62.0,518.9,496.2,542.1,77.0,4481,70.0,NaN
8,All Causes,9,Edison Park,520,65.0,104,928.5,848.7,1008.3,24.0,649.9,593.9,708.4,66.0,5554,59.0,NaN
9,All Causes,10,Norwood Park,2372,7.0,474,1276.9,1225.5,1328.3,4.0,699.6,669.5,730.4,52.0,5994,52.0,NaN


In [72]:
deaths = deaths.dropna()

In [73]:
len(deaths) - deaths.count()

Cause of Death                                                         0
Community Area                                                         0
Community Area Name                                                    0
Cumulative Deaths 2006 - 2010                                          0
Cumulative Deaths Rank                                                 0
Average Annual Deaths 2006 - 2010                                      0
Average Crude Rate 2006 - 2010                                         0
Average Crude Rate Lower CI                                            0
Average Crude Rate Upper CI                                            0
Crude Rate Rank                                                        0
Average Adjusted Rate 2006 - 2010                                      0
Average Adjusted Rate Lower CI                                         0
Average Adjusted Rate Upper CI                                         0
Adjusted Rate Rank                                 

In [74]:
deaths['Cause of Death'].value_counts()

Suicide (intentional self-harm)                                 64
Prostate cancer in males                                        49
Liver disease and cirrhosis                                     48
Firearm-related                                                 47
Assault (homicide)                                              43
Alzheimers disease                                              41
Breast cancer in females                                        36
Colorectal cancer                                               24
Kidney disease (nephritis, nephrotic syndrome and nephrosis)    22
Diabetes-related                                                17
Injury, unintentional                                           16
Stroke (cerebrovascular disease)                                 5
Lung cancer                                                      5
Name: Cause of Death, dtype: int64

In [86]:
import geopandas